In [3]:
import requests # type: ignore

def fetch_all_pairs(
    base_url: str = "https://dlmm-api.meteora.ag",
    include_unknown: bool = True,
    page: int = 0,
    limit: int = 50,
    hide_low_tvl: float = 0.0
):
    """
    Fetches all pairs using the /pair/all_with_pagination endpoint.
    Adjust the query parameters as needed (e.g., sort_key, hide_low_tvl).
    """
    endpoint = f"{base_url}/pair/all_with_pagination"
    params = {
        "include_unknown": str(include_unknown).lower(),  # "true" or "false"
        "page": page,
        "limit": limit,
        "hide_low_tvl": hide_low_tvl,  # filter out extremely low TVL
        # You could also add 'sort_key': 'volume' or 'feetvlratio' if you want server-side sorting
        # 'sort_key': 'feetvlratio',
        # 'order_by': 'desc',
    }
    
    resp = requests.get(endpoint, params=params, timeout=10)
    resp.raise_for_status()  # Raise an exception if the request failed
    data = resp.json()       # Should match the AllPairsWithPagination schema
    return data


def screen_pairs_for_high_fees(pairs_data, min_tvl=5000.0, fee_window="hour_1", min_fee_tvl_ratio=2):
    """
    Screens pairs for:
      1) TVL above 'min_tvl'
      2) Fee/TVL ratio above 'min_fee_tvl_ratio' in the chosen 'fee_window'
         (like 'hour_1', 'hour_2', 'min_30', etc.)
    
    :param pairs_data: list of PairInfo objects
    :param min_tvl: minimum TVL in USD to consider
    :param fee_window: which timeframe to check in fee_tvl_ratio (e.g., 'min_30', 'hour_1', 'hour_24')
    :param min_fee_tvl_ratio: threshold for the ratio
    :return: filtered list of pools
    """
    filtered = []
    for pair in pairs_data:
        # 'liquidity' is stored as a string, so convert to float
        try:
            tvl = float(pair["liquidity"])
        except ValueError:
            continue  # skip if data is malformed
        
        # Check if the pair has a valid fee ratio structure
        fee_ratio_data = pair.get("fee_tvl_ratio", {})
        fee_ratio = fee_ratio_data.get(fee_window, 0.0)
        
        # Filter conditions:
        if tvl >= min_tvl and fee_ratio >= min_fee_tvl_ratio:
            filtered.append(pair)
    
    # Sort by the chosen fee ratio descending, for convenience
    filtered.sort(key=lambda p: p["fee_tvl_ratio"].get(fee_window, 0.0), reverse=True)
    return filtered


def main():
    # 1. Fetch the pair data
    #    NOTE: Replace 'https://dlmm-api.meteora.ag' with the actual base URL.
    all_pairs_response = fetch_all_pairs(base_url="https://dlmm-api.meteora.ag")
    
    # 2. The response structure should match 'AllPairsWithPagination' -> { "pairs": [...], "total": ... }
    pairs = all_pairs_response.get("pairs", [])
    
    # 3. Screen the pairs
    #    For demonstration, we use min_tvl=5000, fee_window='hour_1', min_fee_tvl_ratio=5.0
    #    Adjust these as desired.
    high_fee_pairs = screen_pairs_for_high_fees(
        pairs_data=pairs,
        min_tvl=5000.0,
        fee_window="hour_1",
        min_fee_tvl_ratio=1.0
    )
    
    # 4. Print out the results
    print("Found the following pairs with strong fee generation:\n")
    for pair in high_fee_pairs:
        tvl_str = pair["liquidity"]
        ratio = pair["fee_tvl_ratio"]["hour_1"]
        name = pair["name"]
        print(f"Pair: {name}, TVL: {tvl_str}, Fee/TVL (1h): {ratio}")


if __name__ == "__main__":
    main()


Found the following pairs with strong fee generation:

Pair: mxyz-SOL, TVL: 132314.7535742476, Fee/TVL (1h): 5.13492964013453
Pair: MELANIA-SOL, TVL: 208499.97956266, Fee/TVL (1h): 2.947784810632298
Pair: broccoli-SOL, TVL: 55037.23561968776, Fee/TVL (1h): 2.8303329945871116
Pair: broccoli-SOL, TVL: 80520.4226574124, Fee/TVL (1h): 1.2368920285411482
